In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Apr 28 07:08:16 2024


@author: Faiza
"""

from abc import ABC, abstractmethod
import numpy as np
import struct
from typing import Tuple
from bitarray import bitarray
# from src.compressors.compressor import Compressor

import pandas as pd

import torch
from tqdm import tqdm
from load_dataset import Dataset
import os

import time
# import matplotlib.pyplot as plt
# from PIL import Image
import cv2
import sys
from datetime import datetime


import math
# from src.compressors.compressor import Compressor
import struct


from bitarray import bitarray
from bitarray.util import int2ba, ba2int
np.random.seed(786)

In [2]:


class Compressor(ABC):
    @abstractmethod
    def encode(self, array: np.ndarray) -> bytes:
        pass
    @abstractmethod
    def decode(self, array: bytes) -> np.ndarray:
        pass


# Existing Quantization schemes

In [3]:
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/qsgd.jl")


class QSGD(Compressor):
    def __init__(self, s: int, zero_rle: bool, type=None):
        np.random.seed(16841351)
        self.type = type
        if self.type == "LFL":
            self.s = 2
        else:
            self.s = s
        self.zero_rle = zero_rle

    # format:    | length | norm | s(1) | sign(1) | s(2) | ... | sign(n) | 
    # (no 0-rle) | 32     | 32   | ?    | 1       | ?    | ... | 1       |
    # format:    | length | norm | n_zeros | s(1) | sign(1) | n_zeros | s(2) | ... | sign(n) |
    # (0-rle)    | 32     | 32   | ?       | ?    | 1       | ?       | ?    | ... | 1       |
    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_qsgd(array, self.s, self.type, seed, cid, client_name)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_qsgd(array, self.s, self.type, use_lo_quant)
        return result
    
#%% testing qsgd quantization

print("QSGD")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = QSGD(2,True)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")



e:\MS_Thesis\Hierarchical_quantization\FedPAQ-MNIST-implemenation-main\FedPAQ_env\lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.10.4+0.x64.w64.mingw32. You currently have `1.10.2+0.x64.w64.mingw32` installed. Run:

  juliaup update

in your terminal shell to install Julia 1.10.4+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


QSGD
Before Quantization:  [7.5668163  3.074451   8.939189   3.866898   8.738546   0.76235867
 3.765722   3.2655404  9.408178   4.831219   5.5546727  3.34706
 2.8331633  5.110989   5.5942545  6.458654  ]
Encoded b'(\x00\x00\x00\x114\xb7Ar)E&\x05\x00\x00\x00'
After Quantization:  [11.450212  0.        0.        0.       11.450212  0.       11.450212
  0.       11.450212 11.450212  0.        0.       11.450212  0.
 11.450212  0.      ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [4]:
#%%SETTING UP Julia and gzip Class
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/gzip.jl")

import numpy as np

class GZip(Compressor):
    def __init__(self, s: int):
        np.random.seed(786)
        self.s = s

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_gzip(array, self.s, seed)
        return bytes(result)

    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_gzip(array, self.s)
        return result


#%% testing gzip quantization
print("GZip")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = GZip(1)
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


GZip
Before Quantization:  [7.819443  5.889166  6.3414087 5.9300547 6.9216805 6.8079643 6.2213473
 8.728382  7.2029533 9.9790745 9.917385  0.910352  6.12871   4.920648
 4.4214873 6.2949104]
Encoded b'\x1f\x8b\x08\x00\x00\x00\x00\x00\x00\ncd\x00\x01F0\x02\x81\xe5\xf1\xb7\x1d\x01\x9e\xbbx\xe2\x14\x00\x00\x00'
After Quantization:  [27.421705  0.        0.        0.        0.        0.       27.421705
  0.        0.       27.421705  0.        0.        0.        0.
  0.        0.      ]
38.46153846153846% smaller 
61.904761904761905% smaller 


In [5]:
#%%FP8

# print("Starting up Julia.")
from julia.api import Julia
jl = Julia(compiled_modules=False)
from julia import Main
Main.eval("using Random; Random.seed!(0)")
Main.include("src/compressors/fp8.jl")
# print("Finished starting up Julia.")

FP8_FORMAT = (1, 5, 2)
FP32_FORMAT = (1, 8, 23)

def get_emax(format):
    return (2**(format[1]-1)) - 1

def get_emin(format):
    return 1 - get_emax(format)


class FP8(Compressor):
    def __init__(self):
        np.random.seed(786)
        self.fp8s_repr_in_fp32 = []
        self.fp8s = []
        self.s = -1
        # negative values before positive values.
        def insert(num):
            byte = struct.pack('>B', num)
            [num] = self.decode(byte)
            if not np.isnan(num):
                self.fp8s.append(byte)
                self.fp8s_repr_in_fp32.append(num)
                bits = bitarray()
                bits.frombytes(byte)
        for i in list(reversed(range(128, 253))) + list(range(0, 128)):
            insert(i)
        self.fp8s_repr_in_fp32 = np.array(self.fp8s_repr_in_fp32).astype(np.float32)

    def get_fp8_neighbors(self, f: np.float32) -> Tuple[bytes, bytes]:
        idx_high = np.searchsorted(self.fp8s_repr_in_fp32, f, side='right')
        idx_low = idx_high - 1
        if idx_high == len(self.fp8s_repr_in_fp32):
            idx_high -= 1
        return self.fp8s[idx_low], self.fp8s[idx_high], self.fp8s_repr_in_fp32[idx_low], self.fp8s_repr_in_fp32[idx_high]

    def encode(self, array: np.ndarray, seed=None, cid=":-)", client_name=":-))") -> bytes:
        assert len(array.shape) == 1
        assert array.dtype == np.float32
        result = Main.encode_fp8(array, self.fp8s_repr_in_fp32, self.fp8s, seed)
        return bytes(result)
    
    def decode(self, array: bytes, use_lo_quant=False) -> np.ndarray:
        result = Main.decode_fp8(array)
        return result
#%% testing fp32 quantization
print("FP8")
arr = list(np.random.rand(1,16).flatten()*10)
arr = np.array(arr, dtype = np.float32)
print("Before Quantization: ", arr)
cmp = FP8()
enc = cmp.encode(arr)
print("Encoded",enc)
dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{100 * (sys.getsizeof(arr)-sys.getsizeof(enc))/sys.getsizeof(arr)}% smaller ")


FP8
Before Quantization:  [7.5668163  3.074451   8.939189   3.866898   8.738546   0.76235867
 3.765722   3.2655404  9.408178   4.831219   5.5546727  3.34706
 2.8331633  5.110989   5.5942545  6.458654  ]
Encoded b'HBHDH:DBIEEBBEFF'
After Quantization:  [ 8.    3.    8.    4.    8.    0.75  4.    3.   10.    5.    5.    3.
  3.    5.    6.    6.  ]
52.88461538461539% smaller 
70.83333333333333% smaller 


In [19]:

import cv2


class Custom_linear():
    def __init__(self, quantization_type = np.uint32):
        np.random.seed(348543)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    
class Custom_linear_16():
    def __init__(self, quantization_type = np.uint16):
        np.random.seed(786)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec


#%%

# data = np.random.rand(1,16)

data = list(np.random.rand(1,16).flatten()*10)
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint8
cmp = Custom_linear_16(data_type)
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
# print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))


[2.028032   6.5649495  2.002746   0.10394114 3.833191   4.9089465
 8.527781   3.9331212  0.81519234 2.3411171  1.7995436  3.8348877
 1.0843034  8.128075   1.651963   8.148211  ]
Encoded [ 58 195  57   0 112 145 255 115  21  67  51 112  29 242  46 243]
After Quantization:  [2.01995167 6.54570068 1.98691701 0.10394114 3.80382354 4.89396746
 8.52778053 3.90292753 0.79766909 2.31726365 1.78870902 3.80382354
 1.06194641 8.0983299  1.6235357  8.13136456]
<class 'numpy.uint8'> 48.275862068965516% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  0.00044498449599311886


In [7]:
print(sys.getsizeof(data))
print(sys.getsizeof(dec))
print(sys.getsizeof(enc))

168
232
120


# Custom Frequency Domain Quantization

In [8]:
from scipy.fftpack import dct, idct

class customFreq_1_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        np.random.seed(786)
        self.precision_levels = precision_levels
        
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.7566816  0.3074451  0.8939189  0.3866898  0.87385464 0.07623586
 0.3765722  0.32655403 0.9408179  0.48312193 0.55546725 0.334706
 0.28331634 0.5110989  0.5594255  0.6458654 ]
Encoded [0.9644 0.3823 1.071  0.5923 0.9175 0.3452 0.346  0.4915 0.8613 0.4775
 0.2651 0.2072 0.101  0.3909 0.2537 0.1909]
After Quantization:  [0.75657487 0.30746344 0.89398825 0.38658762 0.8740002  0.07611474
 0.3765901  0.32644805 0.9406408  0.4829445  0.5554271  0.3346513
 0.28326702 0.5111246  0.5593281  0.64587265]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  9.65502e-09


In [9]:
from scipy.fftpack import dct, idct

class customFreq_singleDCT():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        np.random.seed(786)
        self.precision_levels = precision_levels
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        mean_importance = np.mean(importance)
        if mean_importance > 0.1:
            precision = self.precision_levels[0]  # 8-bit
        elif mean_importance > 0.01:
            precision = self.precision_levels[1]  # 4-bit
        else:
            precision = self.precision_levels[2]  # 2-bit
        
        # Quantize frequency components
        enc = self.quantize(weights_f, precision)
        enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.inverse_dct_transform(enc)
        return weights_quantized
    

data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_singleDCT()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)
print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")

from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.7566816  0.3074451  0.8939189  0.3866898  0.87385464 0.07623586
 0.3765722  0.32655403 0.9408179  0.48312193 0.55546725 0.334706
 0.28331634 0.5110989  0.5594255  0.6458654 ]
Encoded [ 2.078    0.11017  0.1637   0.0697   0.2166  -0.4165  -0.1231   0.2578
  0.1963   0.05832 -0.08093  0.07806  0.2084   0.072    0.4219   0.5474 ]
After Quantization:  [0.7566943  0.3074998  0.8939637  0.3867146  0.8738843  0.07625042
 0.37666655 0.32655337 0.94095886 0.4831015  0.55563855 0.33468464
 0.28341928 0.51103944 0.5595117  0.6459183 ]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  5.639836e-09


# Custom Freq 3 (CustomFreq + Custom2 but more sophisticated)

In [10]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        np.random.seed(786)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)
        return enc
    
    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_3():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [np.uint32, np.uint16, np.uint8]):
        np.random.seed(786)
        self.precision_levels = precision_levels
        self.linearQuant_0 = Linear_quant(self.precision_levels[0])
        self.linearQuant_1 = Linear_quant(self.precision_levels[1])
        self.linearQuant_2 = Linear_quant(self.precision_levels[2])
        
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        
        # Calculate importance as the magnitude of the frequency components
        importance = np.abs(weights_f)
        
        # Assign precision based on importance
        self.mean_importance = np.mean(importance)
        # if self.mean_importance > 0.1:
        enc = self.linearQuant_0.encode(weights_f)  # 8-bit
        precision = int(str(self.linearQuant_0.quantization_type).split('t')[-1].split('\'')[0])
        print(precision)

        # elif self.mean_importance > 0.01:
        #     enc = self.linearQuant_1.encode(weights_f)   # 4-bit
        #     precision = int(str(self.linearQuant_1.quantization_type).split('t')[-1].split('\'')[0])
        #     # enc = self.quantize(weights_f, 16)
        # else:
        #     enc = self.linearQuant_2.encode(weights_f)   # 2-bit
        #     precision = int(str(self.linearQuant_2.quantization_type).split('t')[-1].split('\'')[0])

        
        # Quantize frequency components
        # enc = self.quantize(enc, precision)
        return enc


    def decode(self, enc) :

        # if self.mean_importance > 0.1:
        weights_quantized = self.linearQuant_0.decode(enc)  # 8-bit
        # elif self.mean_importance > 0.01:
        #     weights_quantized = self.linearQuant_1.decode(enc)   # 4-bit
        # else:
        #     weights_quantized = self.linearQuant_2.decode(enc)   # 2-bit
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_3()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.7566816  0.3074451  0.8939189  0.3866898  0.87385464 0.07623586
 0.3765722  0.32655403 0.9408179  0.48312193 0.55546725 0.334706
 0.28331634 0.5110989  0.5594255  0.6458654 ]
32
Encoded [3822393318 1245889824 4294967295 2175506503 3614236915 1081460852
 1084719233 1728903523 3366925843 1667425989  726524250  469755745
          0 1282924090  676045071  397722513]
After Quantization:  [0.75668158 0.30744515 0.89391888 0.38668986 0.87385463 0.07623591
 0.37657221 0.32655406 0.94081779 0.48312199 0.55546728 0.33470603
 0.28331629 0.51109892 0.55942545 0.64586541]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.7853008299711994e-15


# CustomFreq2.5

In [11]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        np.random.seed(786)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2_5():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        np.random.seed(786)
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(x, axis=-1, norm='ortho')
    def inverse_dct_transform(self, x):
        return idct(x, axis=-1, norm='ortho')
    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_1_5()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.7566816  0.3074451  0.8939189  0.3866898  0.87385464 0.07623586
 0.3765722  0.32655403 0.9408179  0.48312193 0.55546725 0.334706
 0.28331634 0.5110989  0.5594255  0.6458654 ]
Encoded [0.9644 0.3823 1.071  0.5923 0.9175 0.3452 0.346  0.4915 0.8613 0.4775
 0.2651 0.2072 0.101  0.3909 0.2537 0.1909]
After Quantization:  [0.75657487 0.30746344 0.89398825 0.38658762 0.8740002  0.07611474
 0.3765901  0.32644805 0.9406408  0.4829445  0.5554271  0.3346513
 0.28326702 0.5111246  0.5593281  0.64587265]
<class 'numpy.uint16'> 19.047619047619047% smaller 
<class 'numpy.uint16'> 19.047619047619047% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  9.65502e-09


# Custom Freq 2 (CustomFreq + Custom2) 
- best version

In [12]:
from scipy.fftpack import dct, idct

class Linear_quant():
    def __init__(self, quantization_type = np.uint32):
        np.random.seed(4644684)
        self.quantization_type = quantization_type
        #quantization_level from 0 to 100
        


    def encode(self,data):
        # data = data.astype(int)
        self.xp = [data.min(), data.max()]
        min = np.iinfo(self.quantization_type).min
        max = np.iinfo(self.quantization_type).max

        self.fp = [min, max]
        enc = np.interp(data, self.xp, self.fp)
        enc = enc.astype(self.quantization_type)
        # encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), self.quantization_level] 
        # result, encimg = cv2.imencode('.jpg', img, encode_param)

        return enc

    def decode(self, enc) :
        dec = np.interp(enc, self.fp, self.xp)
        return dec
    

class customFreq_2():
    # def __init__(self, precision_levels = [8, 4, 2]):
    def __init__(self, precision_levels = [32, 16, 8]):
        np.random.seed(786)
        self.precision_levels = precision_levels
        self.linearQuant = Linear_quant()
        #quantization_level from 0 to 100        

    def dct_transform(self, x):
        return dct(dct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def inverse_dct_transform(self, x):
        return idct(idct(x, axis=-1, norm='ortho'), axis=-1, norm='ortho')

    def quantize(self, x, precision):
        scale = 2 ** (precision - 1) - 1
        return np.round(x * scale) / scale


    def encode(self, weights):
        # Apply DCT to transform weights to the frequency domain
        weights_f = self.dct_transform(weights)
        # Quantize frequency components
        enc = self.linearQuant.encode(weights_f)
        # enc = enc.astype(np.float16)
        return enc

    def decode(self, enc) :
        weights_quantized = self.linearQuant.decode(enc)
        weights_quantized = self.inverse_dct_transform(weights_quantized)
        return weights_quantized
    
    
data = list(np.random.rand(1,16).flatten())
data = np.array(data, dtype = np.float32)

print(data)
data_type = np.uint16
cmp = customFreq_2()
enc = cmp.encode(data) #quantization_level from 0 to 100
print("Encoded",enc)

dec = cmp.decode(enc)
print("After Quantization: ", dec)

print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")
print(f"{data_type} {100 * (sys.getsizeof(data)-sys.getsizeof(enc))/sys.getsizeof(data)}% smaller ")


from sklearn.metrics import mean_squared_error as mse
# from scipy.spatial.distance import mse
print("enc shape: ", enc.shape)

print("dec shape: ", dec.shape)

print("mse: ",mse(dec.flatten(), data.flatten()))

[0.7566816  0.3074451  0.8939189  0.3866898  0.87385464 0.07623586
 0.3765722  0.32655403 0.9408179  0.48312193 0.55546725 0.334706
 0.28331634 0.5110989  0.5594255  0.6458654 ]
Encoded [3822393318 1245889824 4294967295 2175506503 3614236915 1081460852
 1084719233 1728903523 3366925843 1667425989  726524250  469755745
          0 1282924090  676045071  397722513]
After Quantization:  [0.75668158 0.30744515 0.89391888 0.38668986 0.87385463 0.07623591
 0.37657221 0.32655406 0.94081779 0.48312199 0.55546728 0.33470603
 0.28331629 0.51109892 0.55942545 0.64586541]
<class 'numpy.uint16'> 27.586206896551722% smaller 
<class 'numpy.uint16'> 0.0% smaller 
enc shape:  (16,)
dec shape:  (16,)
mse:  1.7853008299711994e-15


In [13]:
def get_efficiency(data_size, cmp):
    data = list(np.random.rand(data_size[0],data_size[1]).flatten())
    data = np.array(data, dtype = np.float32)

    print(data)
    data_type = np.uint16

    enc = cmp.encode(data) #quantization_level from 0 to 100
    print("Encoded",enc)

    dec = cmp.decode(enc)
    print("After Quantization: ", dec)

    print(f"{data_type} {100 * (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)}% smaller ")

    eff = (sys.getsizeof(dec)-sys.getsizeof(enc))/sys.getsizeof(dec)


    return eff

# cmp = customFreq_2()

# cmp = FP8()
# cmp = QSGD(2,True)
cmp = Custom_linear()

efficiency = get_efficiency((1,16),cmp)
efficiency


[0.7566816  0.3074451  0.8939189  0.3866898  0.87385464 0.07623586
 0.3765722  0.32655403 0.9408179  0.48312193 0.55546725 0.334706
 0.28331634 0.5110989  0.5594255  0.6458654 ]
Encoded [3380237131 1148573672 4061987944 1542236038 3962315237          0
 1491974933 1243500688 4294967295 2021279997 2380668414 1283997111
 1028709678 2160260743 2400331613 2829737455]
After Quantization:  [0.75668162 0.30744511 0.89391887 0.38668981 0.87385464 0.07623586
 0.37657219 0.32655403 0.94081789 0.48312193 0.55546725 0.33470601
 0.28331634 0.51109892 0.55942547 0.64586538]
<class 'numpy.uint16'> 27.586206896551722% smaller 


0.27586206896551724

# Federated Implementation

In [14]:
#%% PAQ with Compression


class MNIST_PAQ_COMP:
	def __init__(self, cmp, filename="saved_models", number_of_clients=1, aggregate_epochs=10, local_epochs=5, precision=7, r=1.0):
		np.random.seed(786)
		self.model = None
		self.criterion = torch.nn.CrossEntropyLoss()
		self.optimizer = None
		self.number_of_clients = number_of_clients
		self.aggregate_epochs = aggregate_epochs
		self.local_epochs = local_epochs
		self.precision = precision
		self.r = r
		self.filename = filename
		self.compression_ratio = 0
		self.cmp=cmp
		self.cell_column_names = ['scheme_name','Epoch','Loss','Acc']
		self.cell = np.zeros((aggregate_epochs, len(self.cell_column_names)), dtype=object)

	def define_model(self):
		self.model = torch.nn.Sequential(
			torch.nn.Conv2d(1, 2, kernel_size=5),
			torch.nn.ReLU(),
			torch.nn.Conv2d(2, 4, kernel_size=7),
			torch.nn.ReLU(),
			torch.nn.Flatten(),
			torch.nn.Linear(1296, 512),
			torch.nn.ReLU(),
			torch.nn.Linear(512, 128),
			torch.nn.ReLU(),
			torch.nn.Linear(128, 32),
			torch.nn.ReLU(),
			torch.nn.Linear(32, 10),
			torch.nn.Softmax(dim=1),
		)		

	def get_weights_custom(self, dtype=np.float32):
        
		precision = self.precision 
		weights = []
		start_size = 0
		end_size = 0
        
        
		for layer in self.model:			
			try:
				layer_weights = layer.weight.detach().numpy().astype(dtype)
				start_size = start_size + sys.getsizeof(layer_weights)
				# layer_weights_enc = self.cmp.encode(layer_weights)
				layer_weights_enc = self.cmp.encode(layer_weights.flatten())
				decoded_weights = self.cmp.decode(layer_weights_enc)
				decoded_weights = decoded_weights.astype(dtype)
				decoded_weights = decoded_weights.reshape(layer_weights.shape)
				end_size = end_size + sys.getsizeof(decoded_weights)
		
				layer_bias = layer.bias.detach().numpy().astype(dtype)
				# layer_bias_enc = self.cmp.encode(layer_bias)	
				layer_bias_enc = self.cmp.encode(layer_bias.flatten())			
				decoded_bias = self.cmp.decode(layer_bias_enc)
				decoded_bias = decoded_bias.astype(dtype)
				decoded_bias = decoded_bias.reshape(layer_bias.shape)
				weights.append([decoded_weights,decoded_bias])
				
			except:
				continue
		self.compression_ratio = ((start_size-end_size)/start_size)*100
		return np.array(weights),start_size,end_size



	def set_weights(self, weights):
		index = 0
		for layer_no, layer in enumerate(self.model):
			try:
				_ = self.model[layer_no].weight
				self.model[layer_no].weight = torch.nn.Parameter(weights[index][0])
				self.model[layer_no].bias = torch.nn.Parameter(weights[index][1])
				index += 1
			except:
				continue

	def average_weights(self, all_weights):
        
		all_weights = np.array(all_weights)
		all_weights = np.mean(all_weights, axis=0)
		all_weights = [[torch.from_numpy(i[0].astype(np.float32)), torch.from_numpy(i[1].astype(np.float32))] for i in all_weights]
		return all_weights



	def client_generator(self, train_x, train_y):
		number_of_clients = self.number_of_clients
		size = train_y.shape[0]//number_of_clients
		train_x, train_y = train_x.numpy(), train_y.numpy()
		train_x = np.array([train_x[i:i+size] for i in range(0, len(train_x)-len(train_x)%size, size)])
		train_y = np.array([train_y[i:i+size] for i in range(0, len(train_y)-len(train_y)%size, size)])
		train_x = torch.from_numpy(train_x)
		train_y = torch.from_numpy(train_y)
		return train_x, train_y

	def single_client(self, dataset, weights, E):
		self.define_model()
		if weights is not None:
			self.set_weights(weights)
		self.optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
		for epoch in range(E):
			running_loss = 0
			for batch_x, target in zip(dataset['x'], dataset['y']):
				output = self.model(batch_x)
				loss = self.criterion(output, target)
				self.optimizer.zero_grad()
				loss.backward()
				self.optimizer.step()
				running_loss += loss.item()
			running_loss /= len(dataset['y'])
		# weights,start_size,end_size = self.get_weights()
		weights,start_size,end_size = self.get_weights_custom()
		
		return weights, running_loss, start_size,end_size
	def test_aggregated_model(self, test_x, test_y, epoch):
		acc = 0
		with torch.no_grad():
			for batch_x, batch_y in zip(test_x, test_y):
				y_pred = self.model(batch_x)
				y_pred = torch.argmax(y_pred, dim=1)
				acc += torch.sum(y_pred == batch_y)/y_pred.shape[0]
		torch.save(self.model, "./"+self.filename+"/model_epoch_"+str(epoch+1)+".pt")
		return (acc/test_x.shape[0])
			

	def train_aggregator(self, datasets, datasets_test):
		local_epochs = self.local_epochs
		aggregate_epochs = self.aggregate_epochs
		os.system('mkdir '+self.filename)
		E = local_epochs
		aggregate_weights = None
		for epoch in range(aggregate_epochs):
			all_weights = []
			client = 0
			running_loss = 0
			selections = np.arange(datasets['x'].shape[0])
			np.random.shuffle(selections)
			selections = selections[:int(self.r*datasets['x'].shape[0])]
			clients = tqdm(zip(datasets['x'][selections], datasets['y'][selections]), total=selections.shape[0])
			for dataset_x, dataset_y in clients:
				dataset = {'x':dataset_x, 'y':dataset_y}
				weights, loss, start_size,end_size = self.single_client(dataset, aggregate_weights, E)
				running_loss += loss
				all_weights.append(weights)
				client += 1
				clients.set_description(str({"Epoch":epoch+1,"Loss": round(running_loss/client, 5)}))
				clients.refresh()
			aggregate_weights = self.average_weights(all_weights)
			agg_weight = self.set_weights(aggregate_weights)
			test_acc = self.test_aggregated_model(datasets_test['x'], datasets_test['y'], epoch)
			print("Test Accuracy:", round(test_acc.item(), 5))
			print("Compression Ratio ", self.compression_ratio)
			# print()
			self.cell[epoch][0]=self.cmp.__class__.__name__
			self.cell[epoch][1]=epoch
			self.cell[epoch][2]= round(running_loss/client, 3)
			self.cell[epoch][3]= round(test_acc.item(), 3)

			print(f'start_size: {start_size/1024},end_size: {end_size}')
			clients.close()
		return self.cell

#%%

In [20]:
number_of_clients = 328
aggregate_epochs = 10
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()
tic = time.time()
filename = f"saved_models_{epoch_time}"
train_x, train_y, test_x, test_y = Dataset().load_csv()
# cmp = customFreq_2()

# cmp = FP8()
# cmp = QSGD(2,True)
# cmp = Custom_linear_16()
cmp = Custom_linear()
m_8 = MNIST_PAQ_COMP(cmp = cmp, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
train_x, train_y = m_8.client_generator(train_x, train_y)
result = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
print("Time Taken: ", time.time()-tic)


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_19636\592980940.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30101}: 100%|██████████| 164/164 [00:15<00:00, 10.48it/s]


Test Accuracy: 0.10314
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 2, 'Loss': 1.97744}: 100%|██████████| 164/164 [00:16<00:00,  9.98it/s]


Test Accuracy: 0.49276
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 3, 'Loss': 1.73049}: 100%|██████████| 164/164 [00:16<00:00, 10.07it/s]


...

Test Accuracy: 0.85246
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 9, 'Loss': 1.60049}: 100%|██████████| 164/164 [00:16<00:00,  9.66it/s]


Test Accuracy: 0.85705
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784


{'Epoch': 10, 'Loss': 1.61054}: 100%|██████████| 164/164 [00:16<00:00,  9.92it/s]


Test Accuracy: 0.85651
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
Time Taken:  179.46348357200623


In [22]:
result =  pd.DataFrame(result, columns=m_8.cell_column_names)
file_path_1 = r'E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_ACC_LOSS_sheet.csv'
display(result)
if os.path.exists(file_path_1):
    # Read the existing CSV file
    existing_data = pd.read_csv(file_path_1)
    
    # Identify the last epoch in the current run
    last_epoch = result['Epoch'].max()
    
    # Iterate over the new results and update the existing ones if necessary
    for index, row in result.iterrows():
        scheme_name = row['scheme_name']
        epoch = row['Epoch']
        new_acc = row['Acc']
        
        # Check if the scheme_name and epoch already exist in the existing data
        existing_entry = existing_data[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == epoch)]
        
        if not existing_entry.empty:
            # Check if the new accuracy at the last epoch is greater than the existing one
            if epoch == last_epoch:
                existing_last_epoch_entry = existing_data[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == last_epoch)]
                if new_acc > existing_last_epoch_entry['Acc'].values[0]:
                    # Update the existing entry
                    existing_data.loc[(existing_data['scheme_name'] == scheme_name) & (existing_data['Epoch'] == epoch), ['Loss', 'Acc']] = row[['Loss', 'Acc']]
        else:
            # Append the new row to the existing data
            # existing_data = existing_data.append(row, ignore_index=True)
            existing_data = pd.concat([existing_data, pd.DataFrame([row])], ignore_index=True)

    # Save the updated data back to the CSV file
    existing_data.to_csv(file_path_1, index=False)
else:
    # If the file does not exist, save the new result as a new CSV file
    result.to_csv(file_path_1, index=False)


,scheme_name,Epoch,Loss,Acc
0,Custom_linear,0,2.301,0.103
1,Custom_linear,1,1.977,0.493
2,Custom_linear,2,1.73,0.721
3,Custom_linear,3,1.65,0.822
4,Custom_linear,4,1.638,0.833
5,Custom_linear,5,1.621,0.842
6,Custom_linear,6,1.611,0.849
7,Custom_linear,7,1.606,0.852
8,Custom_linear,8,1.6,0.857
9,Custom_linear,9,1.611,0.857


In [23]:
selected_schemes = [Custom_linear(),
                    QSGD(2,True),
                    FP8(),
                    customFreq_2(),
                    Custom_linear_16()]

number_of_clients = 328
aggregate_epochs = 1
local_epochs = 3
r = 0.5
current_time = datetime.now().time()
epoch_time = time.time()

filename = f"saved_models_{epoch_time}"
time_eff_data= []

for scheme in selected_schemes:
    tic = time.time()
    cmp = scheme
    train_x, train_y, test_x, test_y = Dataset().load_csv()
    m_8 = MNIST_PAQ_COMP(cmp = cmp, filename=filename, r=r, number_of_clients=number_of_clients, aggregate_epochs=aggregate_epochs, local_epochs=local_epochs)
    train_x, train_y = m_8.client_generator(train_x, train_y)
    result = m_8.train_aggregator({'x':train_x, 'y':train_y}, {'x':test_x, 'y':test_y})
    # print("Time Taken: ", time.time()-tic)
    time_taken = time.time()-tic
    efficiency = get_efficiency((1,16),cmp)
    time_eff_data.append({'scheme_name':cmp.__class__.__name__,'Time':time_taken,'Efficiency': efficiency})


file_path_time_eff = r'E:\MS_Thesis\Pre-defense-june\Results_for_update\scheme_TIME_EFF_sheet.csv'
time_eff_df = pd.DataFrame(time_eff_data)
time_eff_df.to_csv(file_path_time_eff,index= False)
    

  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_19636\592980940.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30106}: 100%|██████████| 164/164 [00:15<00:00, 10.49it/s]


Test Accuracy: 0.10002
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
[0.48983693 0.25271466 0.74928355 0.6632729  0.47334665 0.6029662
 0.7575445  0.30516094 0.644424   0.9060485  0.30533195 0.45943904
 0.92748994 0.17226228 0.8947991  0.5511899 ]
Encoded [1806041829  457531380 3281510509 2792369438 1712261873 2449406065
 3328490483  755792492 2685176075 4173030065  756764999 1633169515
 4294967295          0 4109055120 2154955153]
After Quantization:  [0.48983693 0.25271466 0.74928355 0.66327292 0.47334665 0.60296619
 0.75754452 0.30516094 0.64442402 0.90604848 0.30533195 0.45943904
 0.92748994 0.17226228 0.89479911 0.5511899 ]
<class 'numpy.uint16'> 27.586206896551722% smaller 


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_19636\592980940.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30088}: 100%|██████████| 164/164 [00:16<00:00,  9.67it/s]


Test Accuracy: 0.10314
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
[0.48983693 0.25271466 0.74928355 0.6632729  0.47334665 0.6029662
 0.7575445  0.30516094 0.644424   0.9060485  0.30533195 0.45943904
 0.92748994 0.17226228 0.8947991  0.5511899 ]
Encoded b'/\x00\x00\x00\xe1\xff\x1d@ED&EJ\x11\x00\x00'
After Quantization:  [0.        1.2343713 1.2343713 1.2343713 1.2343713 0.        0.
 1.2343713 0.        1.2343713 1.2343713 0.        1.2343713 0.
 1.2343713 1.2343713]
<class 'numpy.uint16'> 52.88461538461539% smaller 


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_19636\592980940.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30103}: 100%|██████████| 164/164 [00:24<00:00,  6.57it/s]


Test Accuracy: 0.09963
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
[0.48983693 0.25271466 0.74928355 0.6632729  0.47334665 0.6029662
 0.7575445  0.30516094 0.644424   0.9060485  0.30533195 0.45943904
 0.92748994 0.17226228 0.8947991  0.5511899 ]
Encoded b'84:989:49;58;1;9'
After Quantization:  [0.5     0.25    0.75    0.625   0.5     0.625   0.75    0.25    0.625
 0.875   0.3125  0.5     0.875   0.15625 0.875   0.625  ]
<class 'numpy.uint16'> 52.88461538461539% smaller 


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_19636\592980940.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30111}: 100%|██████████| 164/164 [00:27<00:00,  6.07it/s]


Test Accuracy: 0.10314
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
[0.48983693 0.25271466 0.74928355 0.6632729  0.47334665 0.6029662
 0.7575445  0.30516094 0.644424   0.9060485  0.30533195 0.45943904
 0.92748994 0.17226228 0.8947991  0.5511899 ]
Encoded [2957125735 2355511937 3694611561 4294967295 3251099026 3149377321
 3990055153 2224803502 2813630418 3808821126 1277267491 1624992495
 3186205460          0 2574772618  426744763]
After Quantization:  [0.48983697 0.25271469 0.74928361 0.66327289 0.47334659 0.60296632
 0.75754455 0.30516092 0.64442399 0.9060485  0.30533207 0.45943908
 0.9274899  0.17226235 0.89479908 0.55118981]
<class 'numpy.uint16'> 27.586206896551722% smaller 


  0%|          | 0/164 [00:00<?, ?it/s]C:\Users\Asus\AppData\Local\Temp\ipykernel_19636\592980940.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(weights),start_size,end_size
{'Epoch': 1, 'Loss': 2.30112}: 100%|██████████| 164/164 [00:16<00:00, 10.07it/s]


Test Accuracy: 0.10314
Compression Ratio  99.97330216770052
start_size: 2867.7421875,end_size: 784
[0.48983693 0.25271466 0.74928355 0.6632729  0.47334665 0.6029662
 0.7575445  0.30516094 0.644424   0.9060485  0.30533195 0.45943904
 0.92748994 0.17226228 0.8947991  0.5511899 ]
Encoded [27557  6981 50071 42607 26126 37374 50787 11532 40971 63674 11547 24919
 65535     0 62698 32881]
After Quantization:  [0.48983012 0.25271157 0.74928225 0.66326685 0.47333923 0.60296158
 0.75753346 0.30515746 0.64441353 0.90604371 0.30533032 0.45942972
 0.92748994 0.17226228 0.89479625 0.55118409]
<class 'numpy.uint16'> 41.37931034482759% smaller 


,scheme_name,Epoch,Loss,Acc
0,Custom_linear,0,2.301,0.103
1,Custom_linear,1,2.046,0.443
2,Custom_linear,2,1.88,0.604
3,Custom_linear,3,1.83,0.631
4,Custom_linear,4,1.717,0.676
5,Custom_linear,5,1.643,0.82
6,Custom_linear,6,1.629,0.832
7,Custom_linear,7,1.623,0.838
8,Custom_linear,8,1.62,0.839
9,Custom_linear,9,1.615,0.842
